# Sheep chip aligned
I've aligned with megablast the sheep chip probeset to the oldest `OAR4` assembly. I want to compare my results with result I get from dbSNP

In [1]:
from collections import defaultdict

from src.features.smarterdb import global_connection, VariantSheep
from src.data.common import AssemblyConf

import pandas as pd

In [2]:
conn = global_connection()
dbSNP152 = AssemblyConf(version="Oar_v4.0", imported_from="dbSNP152")

First, get my data aligned and set `snp_name` as a index:

In [3]:
results = pd.read_csv("ovinesnp50-genome-assembly-oar-v4-0.csv-GCA_000298735.2_Oar_v4.0_genomic.fna.blastn.csv")
results.set_index("snp_name", inplace=True)
results.head()

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt
snp_name,,,,,,,,,
250506CS3900065000002_1238.1,15,5859890,C/T,T/C,T/C,BOT,forward,C,T
250506CS3900140500001_312.1,23,26243215,C/T,T/C,T/C,BOT,forward,C,T
250506CS3900176800001_906.1,7,81590897,C/T,T/C,T/C,BOT,forward,C,T
250506CS3900211600001_1041.1,16,41363310,G/T,T/G,T/G,BOT,forward,G,T
250506CS3900218700001_1294.1,2,148834939,C/T,T/C,T/C,BOT,forward,C,T


Next load errors: If I can't place a SNP on a chromosome, I will have no position in results table and I will have a reason in this table:

In [4]:
errors = pd.read_csv("ovinesnp50-genome-assembly-oar-v4-0.csv-GCA_000298735.2_Oar_v4.0_genomic.fna.blastn.err")
errors.set_index("snp_name", inplace=True)
errors.head()

,illumina,illumina_strand,reason
snp_name,,,
Contig35697_5761.1,T/C,BOT,No valid alignments after filtering
CR_816.1,T/C,BOT,Too many alignments after filtering
CytB_1406.1,T/C,BOT,No valid alignments after filtering
CytB_1505.1,A/G,TOP,No valid alignments after filtering
CytB_1745.1,T/C,BOT,No valid alignments after filtering


Ok, now get my Sheep variants and focus on *NCBI* data: I could have more variants than *NCBI* if there are probes more recent than dbSNP152:

In [5]:
ncbi_variants = VariantSheep.objects.filter(chip_name="IlluminaOvineSNP50", locations__match=dbSNP152._asdict(), rs_id__exists=True)
ncbi_variants.count()

48224

Ok now extract dbSNP locations from my `ncbi_variants`:

In [6]:
tmp = defaultdict(list)

for variant in ncbi_variants:
    location = variant.get_location(**dbSNP152._asdict())
    tmp["snp_name"].append(variant.name)
    tmp["rs_id"].append(",".join(variant.rs_id))
    tmp["ncbi_chrom"].append(location.chrom)
    tmp["ncbi_position"].append(location.position)
    
    
ncbi_locations = pd.DataFrame.from_dict(tmp)
ncbi_locations.set_index('snp_name', inplace=True)
ncbi_locations.head()

,rs_id,ncbi_chrom,ncbi_position
snp_name,,,
OAR16_74703566.1,"rs406297509,rs1087899539",16,68777502
OAR5_38485386.1,"rs402039066,rs1093088087",5,34727924
OAR5_87409839_X.1,rs119102657,5,79472633
250506CS3900414400001_1178.1,rs119102699,1,103285485
s05189.1,rs159412897,1,121010442


Ok, merge these data in a new dataframe. Get rid of SNPs not in *NCBI*, mind that since `ncbi_position` could have *NA* values, need to be converted as *integer*:

In [7]:
tmp = results.merge(ncbi_locations, how="left", on="snp_name")
ncbi_results = tmp[tmp['ncbi_chrom'].isna() == False].astype({'ncbi_position':'int'})
ncbi_results.head()

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position
snp_name,,,,,,,,,,,,
250506CS3900065000002_1238.1,15,5859890,C/T,T/C,T/C,BOT,forward,C,T,rs55630613,15,5859890
250506CS3900140500001_312.1,23,26243215,C/T,T/C,T/C,BOT,forward,C,T,rs55630642,23,26243215
250506CS3900176800001_906.1,7,81590897,C/T,T/C,T/C,BOT,forward,C,T,rs55630654,7,81590897
250506CS3900211600001_1041.1,16,41363310,G/T,T/G,T/G,BOT,forward,G,T,rs55630658,16,41363310
250506CS3900218700001_1294.1,2,148834939,C/T,T/C,T/C,BOT,forward,C,T,rs55630663,2,148834939


Ok focus on the differences between my alignment and NCBI:

In [8]:
differences = ncbi_results.query("chrom != ncbi_chrom | position != ncbi_position")
differences.head()

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position
snp_name,,,,,,,,,,,,
250506CS3900371000001_1255.1,11,35291133,C/T,T/C,T/C,BOT,forward,T,C,rs417377113,11,35291132
Contig35697_5761.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs407034953,6,18784742
DU172264_319.1,25,20596182,A/G,T/C,A/G,BOT,reverse,G,A,rs55632153,25,20596183
DU175804_598.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs409850824,13,12526490
DU178311_404.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs55631803,6,36768153


In [9]:
differences.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7137 entries, 250506CS3900371000001_1255.1 to s76034.1
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   chrom             7137 non-null   object
 1   position          7137 non-null   int64 
 2   alleles           683 non-null    object
 3   illumina          7137 non-null   object
 4   illumina_forward  683 non-null    object
 5   illumina_strand   7137 non-null   object
 6   strand            683 non-null    object
 7   ref               683 non-null    object
 8   alt               683 non-null    object
 9   rs_id             7137 non-null   object
 10  ncbi_chrom        7137 non-null   object
 11  ncbi_position     7137 non-null   int64 
dtypes: int64(2), object(10)
memory usage: 724.9+ KB


I have ~7000 differences between NCBI, let's focus on different chromosome types:

In [10]:
differences["chrom"].value_counts()

0             6454
1               90
2               78
3               59
7               35
6               33
4               30
8               27
5               27
15              26
9               26
18              24
10              23
17              22
13              19
11              19
X               18
12              17
22              14
16              13
25              12
20              12
14              11
23              11
19              11
21               9
26               8
24               8
KQ726471.1       1
Name: chrom, dtype: int64

Ok, tell me how many SNPs I can't place, while *NCBI* can:

In [11]:
not_placed = differences[differences["chrom"] == '0']
print(f"There are {not_placed.shape[0]} SNPs that I can't map to genome")
not_placed.merge(errors["reason"], how="left", on="snp_name").sort_values("reason")

There are 6454 SNPs that I can't map to genome


,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position,reason
snp_name,,,,,,,,,,,,,
s53053.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs404810128,5,55369772,Allele doesn't match to reference
OAR16_74353353.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,"rs423249957,rs406660933",16,68460436,Allele doesn't match to reference
OAR4_109410038.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs429699161,4,102752787,Allele doesn't match to reference
OAR25_16816914.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs403759845,25,16179914,Allele doesn't match to reference
OAR1_144667765.1,0,0,NaN,A/C,NaN,TOP,NaN,NaN,NaN,rs426201679,1,133538789,Allele doesn't match to reference
...,...,...,...,...,...,...,...,...,...,...,...,...,...
OAR9_25022978.1,0,0,NaN,T/G,NaN,BOT,NaN,NaN,NaN,rs406411361,9,23984485,Too many alignments after filtering
OAR5_2456339.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs399706799,5,2262230,Too many alignments after filtering
OAR3_18807825.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs427111947,3,17226089,Too many alignments after filtering


Well, there are a lot of SNPs I cannot match. Group them by reason:

In [12]:
not_placed.merge(errors["reason"], how="left", on="snp_name")["reason"].value_counts()

No valid alignments after filtering    6363
Allele doesn't match to reference        69
Too many alignments after filtering      22
Name: reason, dtype: int64

In [13]:
tmp = not_placed.merge(errors["reason"], how="left", on="snp_name")
tmp[tmp["reason"] == "No valid alignments after filtering"]

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position,reason
snp_name,,,,,,,,,,,,,
Contig35697_5761.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs407034953,6,18784742,No valid alignments after filtering
DU175804_598.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs409850824,13,12526490,No valid alignments after filtering
DU178311_404.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs55631803,6,36768153,No valid alignments after filtering
DU179070_177.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs55628106,1,111700859,No valid alignments after filtering
DU189586_521.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs405647639,X,64170493,No valid alignments after filtering
...,...,...,...,...,...,...,...,...,...,...,...,...,...
s75898.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs410893602,5,33620418,No valid alignments after filtering
s75909.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs411541334,9,3183463,No valid alignments after filtering
s75942.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs409109470,20,18138315,No valid alignments after filtering


Are then any *SNP* which I map to a different position than ncbi??

In [14]:
different = differences.query("chrom != '0' and ncbi_chrom != '0'")
print(f"There are {different.shape[0]} SNPs that I can map to a different position")
different.merge(errors["reason"], how="left", on="snp_name").sort_values("reason")

There are 673 SNPs that I can map to a different position


,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position,reason
snp_name,,,,,,,,,,,,,
250506CS3900371000001_1255.1,11,35291133,C/T,T/C,T/C,BOT,forward,T,C,rs417377113,11,35291132,NaN
DU172264_319.1,25,20596182,A/G,T/C,A/G,BOT,reverse,G,A,rs55632153,25,20596183,NaN
DU206327_107.1,17,14328515,C/T,T/C,T/C,BOT,forward,C,T,rs417906482,17,14328514,NaN
DU206996_498.1,5,33118535,A/C,T/G,A/C,BOT,reverse,A,C,"rs403872294,rs421000549",5,33118534,NaN
DU325612_517.1,18,25506831,A/G,T/C,A/G,BOT,reverse,G,A,rs416903259,18,25506832,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
s73957.1,6,57531181,C/T,A/G,T/C,TOP,reverse,C,T,rs426824353,6,57531182,NaN
s74305.1,2,231776557,A/G,A/G,A/G,TOP,forward,A,G,rs406895020,2,231776556,NaN
s74353.1,11,61175955,C/T,T/C,T/C,BOT,forward,C,T,rs405211381,11,61175954,NaN
